In [1]:
import numpy as np
from autograd import jacobian
from autograd import numpy as anp 
from scipy.special import digamma

# Exercise 4.10 (v)

In [ ]:
def compute_mle(x):
    ''' Computes the MLE for (p, k) of the negative binomial distribution given a 
        draw x.
        Parameters:
            x ((n, ) np.ndarray) 
        Reutnrs:
            k ( int ): a positive integer for the estimate of k
            p ( float ): a positve value in [0, 1] as the estimate for p 
    '''

    def sample_mean_estimator(x):
        ''' Returns the value of the sample mean
            Paramters:
                x ((n, ) np.ndarray): the sample
            Retunrs: 
                μ: (float): the sample mean
        '''
        n = x.size
        return np.sum(x) / n

    def p_mom(x):
        ''' Returns the method of moments estimator for p
            Paramters:
                x ((n, ) np.ndarray): the sample
            Retunrs: 
                p_mom: (float): the method of moments estimator for p
        '''
        n = x.size
        p_mom = 1 - (n*sample_mean_estimator(x)) / (np.sum(x**2) - n*sample_mean_estimator(x)**2)
        return p_mom

    def k_mom(x):
        ''' Returns the method of moments estimator for k
            Paramters:
                x ((n, ) np.ndarray): the sample
            Retunrs: 
                p_mom: (float): the method of moments estimator for k
        '''

        return (sample_mean_estimator(x) * (1 - p_mom(x))) / (p_mom(x))

    #we now need to find the zeros of the derivatives we took.
    #To do this we can use Newton's Method with the intial points as the
    #Method of moments estimator. 


    #get initial guess for k
    k_init = k_mom(x)
    #get intial guess for p
    p_init = p_mom(x)

    #define the function from the derivatives we took in the hw
    n = x.size
    μ = sample_mean_estimator(x)
    func = lambda p, k: anp.array( [ n*μ/p - n*k/(1-p), n*anp.log(1-p) + n * digamma + anp.sum(digamma(x + k))  ] ) 
    #define jacobian 
    f_jac = jacobian(func)
    #get initial guess
    x0 = np.array([p_init, k_init])
    #comparison array and tolerance
    x1 = np.zeros()
    tol = 1e-10
    #while loop
    while np.linalg.norm(x0-x1) > tol:
        #calculae next iteration
        temp = x0 - np.linalg.inv(f_jac(x0[0], x0[-1]))*func(x0[0], x0[1])
        x0 = x1.copy()
        x1 = temp.copy()

    #return desired values
    return x1[0], x1[-1]
